<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/%E5%AF%B9%E5%A3%B0%E7%BA%B9%E7%9A%84%E4%BA%A4%E5%8F%89%E6%A3%80%E9%AA%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 对凉宫春日片段声纹识别的交叉验证

**Chat凉宫春日**是模仿凉宫春日等一系列动漫人物，使用近似语气、个性和剧情聊天的语言模型，

<details>
  <summary> 本项目由李鲁鲁，冷子昂，闫晨曦，封小洋，scixing，贾曜恺等开发。 </summary>

李鲁鲁发起了项目，并完成了最早的版本，在多个微信群实现了测试。完成了GPT自动生成对话部分。

冷子昂参与了早期Gradio的开发，并且参与了后端和前端的选型。debug和最终上线了app.py

闫晨曦将李鲁鲁的notebook重构为app.py

封小洋进行了中文转日文模型的选型，完成了针对台词抽取图片的工具，（将要完成）和haruhi图片分类器

scixing实践了VITS语音，完成了台词对应的语音抽取，（将要完成）特定人物的语音分类器。

贾曜恺开发了一个前端实现方案，待测试。

</details>

本脚本由李鲁鲁开发，用于验证声纹特征交叉验证的准确率

- [x] 整理之前的GPTData文件夹
- [x] 确认fxy和scixing是否做了这个事情
- [x] 转移声纹和声音文件夹到GoogleDrive
- [x] 解析标注文件夹，将特征整理成feature，label形式
- [x] 进行交叉验证
- [ ] 给出自动阈值

In [1]:
#mount google drive to colab
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/GPTData/HaruhiRoleVoices

'Haruhi ep10.zip'  'Haruhi ep1.zip'   voiceWithCha.zip


In [26]:
#copy all zip to /content
!cp /content/drive/MyDrive/GPTData/HaruhiRoleVoices/*.zip /content/



In [ ]:
#unzip all zip file
!unzip -q "/content/Haruhi ep10.zip"
!unzip -q "/content/Haruhi ep1.zip"
# !unzip "/content/voiceWithCha.zip"

In [6]:
#unzip /content/voiceWithCha.zip into /content/roles quitely
!unzip -q /content/voiceWithCha.zip -d /content/roles

In [16]:
# scan subfolder in /content/roles and save into a list
import os
roles = os.listdir('/content/roles')

# for each role in roles, scan /content/roles/[role] and save into a list of list
roles_list = []
roles_list_full = []
for role in roles:

    sub_list = os.listdir(os.path.join('/content/roles',role))
    roles_list.append(sub_list)

    full_name_list = ['/content/roles/'+role+'/'+file for file in sub_list]
    roles_list_full.append(full_name_list)


In [17]:
print(roles_list[0])

['68_而是依照凉宫同学的思维模式进行思考而得出的结论.wav', '55_就需要到和那些事件相应的场所去才行.wav', '51_但是这里说那些现实主义的话就太煞风景了.wav', '63_换言之  名侦探所到之处必有事件发生.wav', '54_想要遭遇这种非现实的神秘事件.wav', '44_推理故事作品里的名侦探们.wav', '58_的确  现实世界是不会像故事那样的.wav', '31_我有个非常有钱的远房亲戚.wav', '32_他的钱多到可以买下一个无人岛并且在上面盖上别墅.wav', '50_这样的事件只存在于虚构而非现实的故事世界中.wav', '43_那么你想想「名侦探」那三个字吧.wav', '56_因为创作出来的名侦探们都是这样被卷进事件中的.wav', '349_新川  有什么事吗？.wav', '59_但是现在我们所要去的「孤岛」这种地方.wav', '60_一般人都认为是最适合成为杀人事件舞台的地方.wav', '193_是除我们之外应邀前来的客人.wav', '69_也就是说呢  她想尝试一次看看.wav', '37_臣拜领此职.wav', '52_因为凉宫同学似乎就是想要投身于这样的虚构世界中.wav', '67_「名侦探」和「Closed Circle」并不是我所空想出来的.wav', '64_被称为名侦探的人都有能够呼唤事件的超自然能力.wav', '346_怪不得凉宫同学吃过早饭后.wav', '45_不知道为什么会不停地卷入难以理解的事件中.wav', '33_那栋别墅前几天刚刚落成.wav', '66_我有时候也会偶尔认真一回的.wav']


In [21]:
candidate_path = [
    "/content/Haruhi ep1",
    "/content/Haruhi ep10"
]

In [45]:
import pickle
import numpy as np


features = []
labels = []
dim = 0

count = 0

for role in roles:
    print(role,end=' ')

    for file in roles_list[roles.index(role)]:
        # print(file)

        deal_flag = False

        for candidate in candidate_path:

            candidate_fname = os.path.join(candidate,'voice',file)

            if os.path.exists(candidate_fname):
                # print(candidate_fname,'found')
                deal_flag = True
                feature_fname = os.path.join(candidate,'feature',file) + '.pkl'
                break

        if deal_flag == False:
            print('warning!',file,'not found')
            continue

        if not os.path.exists(feature_fname):
            print('warning!',feature_fname,'not found')
            continue

        # pinkle load feature_fname
        with open(feature_fname,'rb') as f:
            feature = pickle.load(f)

        count += 1

        # append numpy array feature into numpy matrix features
        if dim == 0:
            features = feature
            dim = feature.shape[0]
            # print(dim)
        else:
            features = np.vstack((features,feature))

        labels.append(role)

        # print(feature_fname,'found')



    # break


古泉 多丸裕 谷口 新川先生 阿虚妹妹 长门 春日 朝仓 阿虚 森小姐 忘了叫啥的角色 多丸圭一 朝比奈 

feature是一个N*D的numpy矩阵，每行存储了一个D维特征
labels是一个python的list of string，表示每行对应的数据的标签。
我想验证这批数据使用K近邻分类，在10折交叉时的准确率，请用python为我实现。

In [50]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# 创建 K 近邻分类器模型
k = 1  # 假设选择 k 值为 3
knn = KNeighborsClassifier(n_neighbors=k, metric='cosine')

# 将特征转换为 NumPy 数组
features = np.array(features)

# 将标签转换为 NumPy 数组
labels = np.array(labels)

# 计算 10 折交叉验证的准确率
cv_accuracy = cross_val_score(knn, features, labels, cv=5)

# 打印每折交叉验证的准确率
for fold, accuracy in enumerate(cv_accuracy, 1):
    print(f"Fold {fold}: {accuracy}")

# 打印平均准确率
mean_accuracy = np.mean(cv_accuracy)
print(f"Average Accuracy: {mean_accuracy}")

Fold 1: 0.8809523809523809
Fold 2: 0.9047619047619048
Fold 3: 0.8780487804878049
Fold 4: 0.8536585365853658
Fold 5: 0.8536585365853658
Average Accuracy: 0.8742160278745643


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
